Основное задание:
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые. Внимание! выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.

Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод
fit
/
train
;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (
train
,
valid
);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (
train
,
valid
,
test
);
* ЛБ - лидерборд, выборка
assignment_2_test.csv
.

In [10]:
import pandas as pd
import numpy as np
import pickle
import xgboost as xgb

from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import roc_auc_score
from typing import List, Tuple

In [2]:
df = pd.read_csv('../lesson_2/assignment_2_train.csv')
leader_board = pd.read_csv('../lesson_2/assignment_2_test.csv')

In [3]:
df.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
df.shape

(180000, 394)

In [3]:
df.sort_values('TransactionID', ignore_index=True, inplace=True)

In [4]:
target = 'isFraud'
x_columns = [col for col in df.columns if col != target]

In [5]:
object_features = df.select_dtypes(include=[np.object])
print(f"count of object_features {object_features.shape[1]}")

object_features_col = list(object_features.columns)
object_features_col

count of object_features 14


['ProductCD',
 'card4',
 'card6',
 'P_emaildomain',
 'R_emaildomain',
 'M1',
 'M2',
 'M3',
 'M4',
 'M5',
 'M6',
 'M7',
 'M8',
 'M9']

In [6]:
# Обработка категориальных признаков
for feature in object_features:
    namber_val = 0
    for unic_val in df[feature].unique():
        namber_val += 1
        df.loc[df[feature] == unic_val, feature] = namber_val
        leader_board.loc[leader_board[feature] == unic_val, feature] = namber_val
    df[feature] = pd.to_numeric(df[feature])
    leader_board[feature] = pd.to_numeric(leader_board[feature])

Задание 1:

Сделать Hold-Out валидацию с разбиением, размер которого будет адеквтаным, по вашему мнению; разбиение проводить по id-транзакции (
TransactionID
), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [9]:
x_train = df[:round(df.shape[0]*0.8)][x_columns]
y_train = df[:round(df.shape[0]*0.8)][target]
x_valid = df[round(df.shape[0]*0.8):df.shape[0]][x_columns]
y_valid = df[round(df.shape[0]*0.8):df.shape[0]][target]

In [10]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
#     "n_estimators": 100,
    "reg_lambda": 100,
    "max_depth": 10,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

In [12]:
dtrain = xgb.DMatrix(
    data=x_train, label=y_train
)
dvalid = xgb.DMatrix(
    data=x_valid, label=y_valid
)
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=300,
    early_stopping_rounds=20,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=10,
    maximize=True,
)

[0]	train-auc:0.64131	valid-auc:0.63913
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.79264	valid-auc:0.79753
[20]	train-auc:0.86040	valid-auc:0.84586
[30]	train-auc:0.87652	valid-auc:0.85310
[40]	train-auc:0.89047	valid-auc:0.86075
[50]	train-auc:0.90219	valid-auc:0.86481
[60]	train-auc:0.91023	valid-auc:0.87490
[70]	train-auc:0.91608	valid-auc:0.88086
[80]	train-auc:0.91976	valid-auc:0.88455
[90]	train-auc:0.92274	valid-auc:0.88649
[100]	train-auc:0.92517	valid-auc:0.88859
[110]	train-auc:0.92685	valid-auc:0.89054
[120]	train-auc:0.92832	valid-auc:0.89216
[130]	train-auc:0.92931	valid-auc:0.89254
[140]	train-auc:0.92976	valid-auc:0.89300
[150]	train-auc:0.92976	valid-auc:0.89300
Stopping. Best iteration:
[133]	train-auc:0.92976	valid-auc:0.89300



In [14]:
filename = 'xgb_model_1.pkl'
pickle.dump(model, open(filename, 'wb'))

In [17]:
dlid_board = xgb.DMatrix(
    data=leader_board[x_columns], label=leader_board[target]
)
predict_val = model.predict(dlid_board)
roc_auc_score(leader_board[target], predict_val)

0.8702878377416603

Выводы:

В данной ситуации, нельзя сделать однозначные выводы переобучилась модель или нет. С одной стороны score на тренировочной и валидационной выборках больше чем на лидерборде, что может свидетельствовать о переобучении модели. Но разница не очень велика и мы не имеем дополнительной информации о распределении score на валидационной и тренировочной выборках для однозначного ответа.

Задание 2:

Сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [7]:
x_train = df[:round(df.shape[0]*0.8)][x_columns]
y_train = df[:round(df.shape[0]*0.8)][target]
x_valid = df[round(df.shape[0]*0.8):round(df.shape[0]*0.9)][x_columns]
y_valid = df[round(df.shape[0]*0.8):round(df.shape[0]*0.9)][target]
x_test = df[round(df.shape[0]*0.9):df.shape[0]][x_columns]
y_test = df[round(df.shape[0]*0.9):df.shape[0]][target]

In [28]:
dtrain = xgb.DMatrix(
    data=x_train, label=y_train
)
dvalid = xgb.DMatrix(
    data=x_valid, label=y_valid
)
model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=300,
    early_stopping_rounds=20,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    verbose_eval=10,
    maximize=True,
)

[0]	train-auc:0.64131	valid-auc:0.63470
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 20 rounds.
[10]	train-auc:0.79264	valid-auc:0.80565
[20]	train-auc:0.86040	valid-auc:0.85785
[30]	train-auc:0.87652	valid-auc:0.86533
[40]	train-auc:0.89047	valid-auc:0.87233
[50]	train-auc:0.90219	valid-auc:0.87640
[60]	train-auc:0.91023	valid-auc:0.88529
[70]	train-auc:0.91608	valid-auc:0.89012
[80]	train-auc:0.91976	valid-auc:0.89298
[90]	train-auc:0.92274	valid-auc:0.89504
[100]	train-auc:0.92517	valid-auc:0.89735
[110]	train-auc:0.92685	valid-auc:0.89850
[120]	train-auc:0.92832	valid-auc:0.89962
[130]	train-auc:0.92931	valid-auc:0.90015
[140]	train-auc:0.92976	valid-auc:0.90058
[150]	train-auc:0.92976	valid-auc:0.90058
Stopping. Best iteration:
[133]	train-auc:0.92976	valid-auc:0.90058



In [29]:
filename = 'xgb_model_2.pkl'
pickle.dump(model, open(filename, 'wb'))

In [30]:
dtest = xgb.DMatrix(
    data=x_test, label=y_test
)
dlid_board = xgb.DMatrix(
    data=leader_board[x_columns], label=leader_board[target]
)
predict_val_test = model.predict(dtest)
predict_val_lb = model.predict(dlid_board)

metric_test = roc_auc_score(y_test, predict_val_test)
metric_lb = roc_auc_score(leader_board[target], predict_val_lb)

print(f'ROC_AUC for test: {metric_test} \nROC_AUC for lider board: {metric_lb}')

ROC_AUC for test: 0.8858630883567298 
ROC_AUC for lider board: 0.8702878377416603


Выводы:

Ситуации повторяется. Нельзя сделать однозначные выводы переобучилась модель или нет.

Задание 3:

Построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [34]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [35]:
np.random.seed(27)
scores = create_bootstrap_metrics(y_test, predict_val_test, roc_auc_score)

calculate_confidence_interval(scores)

(0.8701854077880476, 0.9005647109737781)

In [36]:
np.random.seed(27)
scores = create_bootstrap_metrics(leader_board[target], predict_val_lb, roc_auc_score)

calculate_confidence_interval(scores)

(0.8634762507957352, 0.876603369553642)

Выводы:

После проведения дополнительного исследования, можно сделать вывод о том что метрика качества находится в пределах доверительного интервала. Поэтому можно сказать что модель обучилась хорошо и ей можно доверять.

Задание 4:

Выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

In [8]:
x_adv = pd.concat([
    x_train, x_valid], axis=0
)
y_adv = np.hstack((np.zeros(x_train.shape[0]), np.ones(x_valid.shape[0])))
assert x_adv.shape[0] == y_adv.shape[0]

In [11]:
model = xgb.XGBClassifier(n_estimators=25)
model.fit(x_adv, y_adv)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=25, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [12]:
y_pred_adv = model.predict_proba(x_adv)
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 4))

1.0


In [14]:
y_pred = model.predict_proba(x_train)
y_pred

array([[9.9974763e-01, 2.5233824e-04],
       [9.9974763e-01, 2.5233824e-04],
       [9.9974763e-01, 2.5233824e-04],
       ...,
       [9.9974763e-01, 2.5233824e-04],
       [9.9974763e-01, 2.5233824e-04],
       [9.9974763e-01, 2.5233824e-04]], dtype=float32)

In [15]:
pd.cut(
    y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]    144000
(0.1, 0.2]         0
(0.2, 0.3]         0
(0.3, 0.4]         0
(0.4, 0.5]         0
(0.5, 0.6]         0
(0.6, 0.7]         0
(0.7, 0.8]         0
(0.8, 0.9]         0
(0.9, 1.0]         0
dtype: int64

Выводы:

Задание 5:

Сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

In [40]:
model_t = xgb.XGBRegressor(
    eval_metric="auc",
    booster="gbtree",
    objective="binary:logistic",
    learning_rate=0.1,
    reg_lambda=100,
    max_depth=10,
    gamma=10,
    seed=27,
    num_boost_round=300,
    early_stopping_rounds=20,
    verbose_eval=10,
    maximize=True,)

model_t.fit(x_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, early_stopping_rounds=20,
             eval_metric='auc', gamma=10, gpu_id=-1, importance_type='gain',
             interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
             max_depth=10, maximize=True, min_child_weight=1, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=0,
             num_boost_round=300, num_parallel_tree=1,
             objective='binary:logistic', random_state=27, reg_alpha=0,
             reg_lambda=100, scale_pos_weight=1, seed=27, subsample=1,
             tree_method=None, ...)

In [43]:
train_score = roc_auc_score(y_train, model_t.predict(x_train))
valid_score = roc_auc_score(y_valid, model_t.predict(x_valid))
test_score = roc_auc_score(y_test, model_t.predict(x_test))

print(f"Train-score: {round(train_score, 3)}, Valid-score: {round(valid_score, 3)}, Test-score: {round(test_score, 3)}")

Train-score: 0.925, Valid-score: 0.897, Test-score: 0.88


In [44]:
cv = cross_val_score(
    estimator=model_t,
    X=df[x_columns],
    y=df[target],
    scoring="roc_auc",
    cv=10
)

print(f"CV-results: {round(np.mean(cv), 4)} +/- {round(np.std(cv), 3)}")

CV-results: 0.904 +/- 0.014


In [45]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          estimator: object,
                          metric: callable,
                          cv_strategy):
    """
    Кросс-валидация.

    Parameters
    ----------
    X: pd.DataFrame
        Матрица признаков.

    y: pd.Series
        Вектор целевой переменной.

    estimator: callable
        Объект модели для обучения.

    metric: callable
        Метрика для оценки качества решения.
        Ожидается, что на вход будет передана функция,
        которая принимает 2 аргумента: y_true, y_pred.

    cv_strategy: cross-validation generator
        Объект для описания стратегии кросс-валидации.
        Ожидается, что на вход будет передан объект типа
        KFold или StratifiedKFold.

    Returns
    -------
    oof_score: float
        Значение метрики качества на OOF-прогнозах.

    fold_train_scores: List[float]
        Значение метрики качества на каждом обучающем датасете кросс-валидации.

    fold_valid_scores: List[float]
        Значение метрики качества на каждом валидационном датасете кросс-валидации.

    oof_predictions: np.array
        Прогнозы на OOF.

    """
    estimators, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        estimator.fit(x_train, y_train)
        y_train_pred = estimator.predict(x_train)
        y_valid_pred = estimator.predict(x_valid)

        fold_train_scores.append(metric(y_train, y_train_pred))
        fold_valid_scores.append(metric(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        estimators.append(estimator)

    oof_score = metric(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [46]:
cv_strategy = KFold(n_splits=5, random_state=1)

estimators, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    df[x_columns], df[target], model_t, metric=roc_auc_score, cv_strategy=cv_strategy
)

/home/aleksandr/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.929, valid-score = 0.8887
Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.9292, valid-score = 0.9025
Fold: 3, train-observations = 144000, valid-observations = 36000
train-score = 0.9248, valid-score = 0.9139
Fold: 4, train-observations = 144000, valid-observations = 36000
train-score = 0.9263, valid-score = 0.8988
Fold: 5, train-observations = 144000, valid-observations = 36000
train-score = 0.925, valid-score = 0.8881
CV-results train: 0.9269 +/- 0.002
CV-results valid: 0.8984 +/- 0.01
OOF-score = 0.8981


Выводы:

После проведения кросс-валидации можно сказать что модель достаточно стабильна. Разброс метрики качества очень мальенький.

Задание 6 (опциональное):

сделать Hold-Out валидацию по времени (TransactionDT), повторить процедуры из п.1 / п.2 (на ваш выбор). Построить доверительный интервал, сравнить качество на ЛБ выборке с полученным доверительным интервалом. Сделать выводы.

Задание 7 (совсем опциональное):

в данном наборе данных у нас есть ID-транзакции (TransactionID) и время транзакции (TransactionDT), но отсутствует ID-клиента, который совершал транзакции. Кажется, что в этой задаче валидация по клиенту работала бы хорошо. Предложить критерий, по которому можно выделить клиентов и сделать п.5, используя созданное определение клиента, используя валидацию по клиенту (GroupKFold).